In [1]:
import openai
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

openai.api_base = "https://openai.vocareum.com/v1"
api_key = "YOUR API KEY"
openai.api_key = api_key

## Generate Dummy Real Estate Data

In [2]:
example = """
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."""

print(example)


Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.


In [3]:
prompt = f"""
    
    Create dummy data in JSON format for 10 real estate listings.
    Each listing should contain the fields
    Neighborhood:
    Price:
    Bedrooms: int 
    Bathrooms: int
    House Size:
    Description: str
    Neighborhood Description: str
    
    An example of this is:
    {example}
    
"""

In [4]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a real estate agent."},
        {"role": "user", "content": f"{prompt}"}
    ],
    response_format={"type": "json_object"},
    temperature=1,
    max_tokens=4000,
    top_p=1,
    )

listings = response["choices"][0]["message"]["content"]


In [5]:
print(listings)

{
    "listings": [
        {
            "Neighborhood": "Riverfront Estates",
            "Price": "$1,200,000",
            "Bedrooms": 4,
            "Bathrooms": 3,
            "House Size": "3,500 sqft",
            "Description": "Escape to the luxurious lifestyle of Riverfront Estates where this exquisite 4-bedroom, 3-bathroom home awaits. Enjoy stunning river views from the expansive windows that fill the house with natural light. The gourmet kitchen features top-of-the-line appliances and opens up to a grand living room with a fireplace. Step outside to the meticulously landscaped backyard and relax by the tranquil waterfall. Live in elegance at Riverfront Estates.",
            "Neighborhood Description": "Riverfront Estates offers upscale living along the riverfront with access to private marinas and exclusive dining experiences. Immerse yourself in the beauty of nature while being just a short drive away from the bustling city center."
        },
        {
            "Nei

In [6]:
import json

listings_list = json.loads(listings)["listings"]
listings_list[:2]

[{'Neighborhood': 'Riverfront Estates',
  'Price': '$1,200,000',
  'Bedrooms': 4,
  'Bathrooms': 3,
  'House Size': '3,500 sqft',
  'Description': 'Escape to the luxurious lifestyle of Riverfront Estates where this exquisite 4-bedroom, 3-bathroom home awaits. Enjoy stunning river views from the expansive windows that fill the house with natural light. The gourmet kitchen features top-of-the-line appliances and opens up to a grand living room with a fireplace. Step outside to the meticulously landscaped backyard and relax by the tranquil waterfall. Live in elegance at Riverfront Estates.',
  'Neighborhood Description': 'Riverfront Estates offers upscale living along the riverfront with access to private marinas and exclusive dining experiences. Immerse yourself in the beauty of nature while being just a short drive away from the bustling city center.'},
 {'Neighborhood': 'Mountain View Heights',
  'Price': '$950,000',
  'Bedrooms': 3,
  'Bathrooms': 2,
  'House Size': '2,200 sqft',
  'D

In [7]:
import pandas as pd

df = pd.DataFrame(listings_list)
df.to_csv("listings.csv", index=False)

df.head(3)

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Riverfront Estates,"$1,200,000",4,3,"3,500 sqft",Escape to the luxurious lifestyle of Riverfron...,Riverfront Estates offers upscale living along...
1,Mountain View Heights,"$950,000",3,2,"2,200 sqft",Embrace mountain living in this charming 3-bed...,Mountain View Heights is known for its picture...
2,Lakefront Haven,"$1,500,000",5,4,"4,000 sqft",Discover the ultimate lakefront living in this...,Lakefront Haven offers an exclusive waterfront...


## Generate Embeddings

In [8]:
from lancedb.pydantic import Vector, LanceModel
from lancedb.embeddings import get_registry

import numpy as np

def combine_columns(row):
    return str("Neighborhood: " + row['Neighborhood']) + ", Price: " + str(row['Price']) + ", Bedrooms: " + str(row['Bedrooms'])\
        + ", Bathrooms: "+ str(row['Bathrooms']) + ", House Size: " + str(row['House Size']) + ", Description: " + row['Description']\
        + ", Neighborhood Description: "+ row['Neighborhood Description']


df['combined'] = df.apply(lambda x:combine_columns(x), axis=1)

In [9]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["combined"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings

In [10]:
df.head(2)

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description,combined,embeddings
0,Riverfront Estates,"$1,200,000",4,3,"3,500 sqft",Escape to the luxurious lifestyle of Riverfron...,Riverfront Estates offers upscale living along...,"Neighborhood: Riverfront Estates, Price: $1,20...","[0.0015989412786439061, 0.01223020814359188, -..."
1,Mountain View Heights,"$950,000",3,2,"2,200 sqft",Embrace mountain living in this charming 3-bed...,Mountain View Heights is known for its picture...,"Neighborhood: Mountain View Heights, Price: $9...","[0.013038930483162403, 0.005120590794831514, -..."


In [11]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

## Filtering by User Requirements

In [12]:
questions = [   
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

In [13]:
userRequirements = " ".join(answers)
sortedRows = get_rows_sorted_by_relevance(userRequirements, df)

In [14]:
relevantRows = sortedRows[sortedRows["distances"] < 0.17].head(5)
relevantRows

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description,combined,embeddings,distances
4,Suburban Meadows,"$800,000",4,3,"2,500 sqft","Welcome home to this spacious 4-bedroom, 3-bat...",Suburban Meadows is a peaceful community with ...,"Neighborhood: Suburban Meadows, Price: $800,00...","[0.006916717626154423, 0.024977391585707664, -...",0.134989
7,Historic Downtown,"$900,000",3,2,"2,300 sqft",Step back in time with this charming 3-bedroom...,Historic Downtown is a vibrant district filled...,"Neighborhood: Historic Downtown, Price: $900,0...","[0.002905286382883787, 0.02250443957746029, -0...",0.168252


In [15]:
matches = [i for i in relevantRows["combined"]]
matches

['Neighborhood: Suburban Meadows, Price: $800,000, Bedrooms: 4, Bathrooms: 3, House Size: 2,500 sqft, Description: Welcome home to this spacious 4-bedroom, 3-bathroom residence in the family-friendly Suburban Meadows neighborhood. The airy layout features a formal living room, cozy family room with a fireplace, and a gourmet kitchen with granite countertops. Retreat to the private backyard oasis complete with a lush lawn and patio for outdoor gatherings. With a designated home office and ample storage space, this home has it all., Neighborhood Description: Suburban Meadows is a peaceful community with tree-lined streets, parks, and top-rated schools. Enjoy a suburban lifestyle with easy access to shopping centers, restaurants, and community events.',
 'Neighborhood: Historic Downtown, Price: $900,000, Bedrooms: 3, Bathrooms: 2, House Size: 2,300 sqft, Description: Step back in time with this charming 3-bedroom, 2-bathroom home located in the Historic Downtown district. The historic fea

## Real Estate Agent Sell Pitch

In [16]:
prompt = f"""
    A buyer is looking for properties with these requirements:
    {userRequirements}

    You are a real estate agent tasked with selling properties.
    For each of the properties in the list below, create a persuasive and engaging sales pitch from the perspective of a knowledgeable and enthusiastic real estate agent.
    The pitch should highlight how each property meets the buyer's needs, focusing on the benefits and lifestyle aspects, not just the specifications.
    Also suggest what doesnt match the user's needs and list how those properties could be beneficial to have
    Do not change any factual information in the properties's spesification
    Use a friendly and professional tone to make the buyer feel confident and excited about the potential of each property.
    Ensure that the descriptions are vivid and relatable, emphasizing the quality of life they can expect from living in these homes.

    Here are the properties:
    {matches}
"""

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a real estate agent."},
        {"role": "user", "content": f"{prompt}"}
    ],
    temperature=1,
    max_tokens=4000,
    top_p=1,
    )

sellPitch = response["choices"][0]["message"]["content"]

In [17]:
print(sellPitch)

Property in Suburban Meadows:
🌟 Welcome to your new sanctuary in the heart of the family-friendly Suburban Meadows neighborhood! This spacious 4-bedroom, 3-bathroom home is perfect for creating lasting memories with your loved ones. The airy layout offers a formal living room for elegant entertaining, a cozy family room with a fireplace for cozy evenings, and a gourmet kitchen with granite countertops where culinary masterpieces come to life.

🏡 Imagine retreating to your own private backyard oasis, complete with a lush lawn and a patio perfect for al fresco dining or relaxing under the stars. Need a designated home office or extra storage space? This home has you covered with ample room for all your needs.

🌳 Suburban Meadows offers a peaceful setting with tree-lined streets, parks, and top-rated schools, making it an ideal place to raise a family. Enjoy the convenience of nearby shopping centers, restaurants, and community events while basking in the tranquil vibe of this well-establ